<title>[Hello]</title>
<img align="center" src="https://inspirebrands.com/wp-content/uploads/2017/10/Inspire_Logo.png" width="280" height="70">

<hr>

# <center><u>Sales Forecast, Print Media Effectiveness, and Marketing Strategies</u></center>

## Introduction

The Brand President and CMO require aid in setting the course for the company's future and evaluating the efficacy of marketing efforts. To accomplish this, they request insight from the brand's data science team.

### Problem Satement

With available data, the executives are requesting answers to the following questions:

1. What are the forecasted, corporate weekly sales?
2. Is the print media program effective? What lift does it provide?
3. What additional (if any) inputs are most descriptive of sales?
4. Given the above findings, what marketing strategies would you shift to increase sales growth?

### Approach

#### Sales Forecast

TBD

#### Print Media

TBD

#### Sales Insight

TBD

#### Marketing Strategies

TBD

### Imports

In [1]:
import numpy as np
import pandas as pd

from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

## Data Processing

The available data are stored in CSV files. To be suitable for evaluation use, they need to be cleansed and aggregated. The following DataFrames are the result of this processing:

1. **arg_sales** - Contains daily sales at location level
    - rest_number (int64)
    - rest_dma_code (int64)
    - rest_dma_desc (object)
    - fiscal_year (int64)
    - fiscal_week_number (int64)
    - days_date (object)
    - sales_ty (float64)
    
2. **media**
    - rest_dma_code (int64)
    - fiscal_year (int64)
    - fiscal_week_number (int64)
    - onair_ty (int64)
    - equal_trps_cm_ty (float64)
    
3. **competitive** - Contains an integer quantity count per competitor for five competitors within a radius of 1 and 0.5 miles
    - store (int64)
    - mcd_1_mi, subway_1_mi, wendys_1_mi, bk_1_mi, tb_1_mi (int64)
    - mcd_hlf_mi, subway_hlf_mi, wendys_hlf_mi, bk_hlf_mi, tb_hlf_mi (int64)
    
4. **restaurant** - Contains store level profile information regarding relative location, store status, open/close/remodel dates
    - rest_number (Int64)
    - rest_status (object)
    - rest_addrstate (object)
    - rest_remodeldt (object)
    - rest_opendate (object)
    - rest_closedate (object)
    
5. **store_print** - Unclear; "drop_date" is the significant value, but don't understand its purpose
    - rest_number (int64)
    - drop_date (object)
    - fiscal_year (int64)
    - fiscal_week_number (int64)
    
6. **weather** - Contains weather data at store level for a given day
    - rest_number (Int64)
    - rest_dma_code (Int64)
    - rest_dma_desc (object)
    - date_ty (object)
    - temp_avg, temp_max, temp_min (float64)
    - thunderstorm_bucket, rain_bucket, snow_bucket (object)
    - snow, thunderstorm, rain, clear_weather (Int64): Boolean values

### arg_sales

*arg_sales* is a DataFrame comprised of two separate CSV files (arg_sales_2015_2016.csv & arg_sales_2017_2018.csv), and they contain daily sales values at the store level with additional location and calendar information.

The following changes are applied to alter their original form:

1. Both CSV files are read, appended, and stored in a final DataFrame
2. The DataFrame index is reset to eliminate duplication values as a result of the concat function
3. Column headers are reduced to lower case
4. Records from location 6127 were removed due to large sales abnormalities (e.g. negative $50k sales in one day)

In [2]:
arg_sales = pd.concat(map(pd.read_csv, ['arg_sales_2015_2016.csv', 'arg_sales_2017_2018.csv']))

In [3]:
arg_sales = arg_sales.reset_index(drop=True)

In [4]:
arg_sales.columns = map(str.lower, arg_sales.columns)

In [5]:
arg_sales.head()

,rest_number,rest_dma_code,rest_dma_desc,fiscal_year,fiscal_week_number,days_date,sales_ty
0,1,536,YOUNGSTOWN,2015,1,12/29/2014,2600.03
1,1,536,YOUNGSTOWN,2015,1,12/30/2014,2333.04
2,1,536,YOUNGSTOWN,2015,1,12/31/2014,2045.05
3,1,536,YOUNGSTOWN,2015,1,1/1/2015,1355.33
4,1,536,YOUNGSTOWN,2015,1,1/2/2015,2645.63


In [6]:
arg_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1495669 entries, 0 to 1495668
Data columns (total 7 columns):
rest_number           1495669 non-null int64
rest_dma_code         1495669 non-null int64
rest_dma_desc         1495669 non-null object
fiscal_year           1495669 non-null int64
fiscal_week_number    1495669 non-null int64
days_date             1495669 non-null object
sales_ty              1495669 non-null float64
dtypes: float64(1), int64(4), object(2)
memory usage: 79.9+ MB


**Note**: No values are missing from any columns.

In [7]:
arg_sales.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
rest_number,1495669.0,4578.443654,2987.946855,1.00,1172.00,5817.00,7205.00,9002.00
rest_dma_code,1495669.0,565.140629,65.725009,501.00,518.00,536.00,613.00,819.00
fiscal_year,1495669.0,2016.564610,1.116557,2015.00,2016.00,2017.00,2018.00,2018.00
fiscal_week_number,1495669.0,26.765992,15.038973,1.00,14.00,27.00,40.00,53.00
sales_ty,1495669.0,3547.334603,1261.547286,-54736.12,2721.49,3442.43,4264.99,57748.24


**Note**: *sale_ty* exhibits extreme low and extreme highs. Further investigation is warranted.

Display record with maximum *sales_ty* vale

In [8]:
arg_sales.loc[arg_sales['sales_ty'].idxmax()]

rest_number                                     6127
rest_dma_code                                    555
rest_dma_desc         SYRACUSE                      
fiscal_year                                     2018
fiscal_week_number                                13
days_date                                  3/29/2018
sales_ty                                     57748.2
Name: 1128654, dtype: object

Display record with minimum *sales_ty* value

In [9]:
arg_sales.loc[arg_sales['sales_ty'].idxmin()]

rest_number                                     6127
rest_dma_code                                    555
rest_dma_desc         SYRACUSE                      
fiscal_year                                     2018
fiscal_week_number                                13
days_date                                   4/1/2018
sales_ty                                    -54736.1
Name: 1128657, dtype: object

**Note**: Both the extreme low and high values come from the same location during the same week. This may be the result of some type of register reconciliation but, without knowing exactly, we'll assume corruption at this store and remove its records even though if aggregated weekly, it will wash out.

Remove all sales records where *rest_number* is *6127*.

In [10]:
arg_sales = (arg_sales[arg_sales['rest_number'] != 6127]).reset_index(drop=True)

In [11]:
arg_sales.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
rest_number,1495371.0,4578.135055,2988.164584,1.00,1172.00,5817.00,7205.00,9002.00
rest_dma_code,1495371.0,565.142650,65.731402,501.00,518.00,536.00,613.00,819.00
fiscal_year,1495371.0,2016.564324,1.116485,2015.00,2016.00,2017.00,2018.00,2018.00
fiscal_week_number,1495371.0,26.765106,15.039339,1.00,14.00,27.00,40.00,53.00
sales_ty,1495371.0,3547.232146,1259.922024,-4579.63,2721.35,3442.26,4264.89,22540.26


**Note**: The daily sales values due still exhibit negative values, but they're no longer extreme. These will be assumed to be natural sales reconciliations and remain in the data.

### media_data

The *media* DataFrame requires very little modification (i.e. column headers modified to lower case).

The DataFrame is a general indication of whether a DMA had on-air media during a given period of time. This is indicated by a binary value and then further described by the degree of media as indicated via a TRP value.

In [12]:
media = pd.read_csv('case_study_media_data.csv')

In [13]:
media.columns = map(str.lower, media.columns)

In [14]:
media.head()

,rest_dma_code,fiscal_year,fiscal_week_number,onair_ty,equal_trps_cm_ty
0,501,2015,1,0,0.0
1,501,2015,2,0,0.0
2,501,2015,3,0,0.0
3,501,2015,4,0,0.0
4,501,2015,5,0,0.0


In [15]:
media.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13728 entries, 0 to 13727
Data columns (total 5 columns):
rest_dma_code         13728 non-null int64
fiscal_year           13728 non-null int64
fiscal_week_number    13728 non-null int64
onair_ty              13728 non-null int64
equal_trps_cm_ty      13728 non-null float64
dtypes: float64(1), int64(4)
memory usage: 536.4 KB


In [16]:
media.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
rest_dma_code,13728.0,572.227273,65.344787,501.0,525.00,550.50,596.00,770.0
fiscal_year,13728.0,2016.500000,1.118075,2015.0,2015.75,2016.50,2017.25,2018.0
fiscal_week_number,13728.0,26.500000,15.008878,1.0,13.75,26.50,39.25,52.0
onair_ty,13728.0,0.816725,0.386906,0.0,1.00,1.00,1.00,1.0
equal_trps_cm_ty,13728.0,148.256957,81.026854,0.0,96.40,155.65,199.80,536.3


### competitive

The *competitive* DataFrame holds values of store counts for (5) perceived competitors within a one-mile and half-mile radius.

The following measures were taken in the processing of the data:

1. The CSV was read and stored in a DataFrame
2. Column headers were renamed
3. Integrity check to ensure the half-mile competitor count is never higher than 1-mile competitor count

In [17]:
competitive = pd.read_csv('competitive_data.csv')

In [18]:
competitive = competitive.rename(columns={'Store #':'store',
                                          '# of McDonalds <1 mi':'mcd_1_mi',
                                          '# of Subway <1 mi':'subway_1_mi',
                                          '# of Wendys <1 mi':'wendys_1_mi',
                                          '# of Burger King <1 mi':'bk_1_mi',
                                          '# of Taco Bell <1 mi':'tb_1_mi',
                                          '# of McDonalds <0.5 mi':'mcd_hlf_mi',
                                          '# of Subway <0.5 mi':'subway_hlf_mi',
                                          '# of Wendys <0.5 mi':'wendys_hlf_mi',
                                          '# of Burger King <0.5 mi':'bk_hlf_mi',
                                          '# of Taco Bell <0.5 mi':'tb_hlf_mi'})

In [19]:
competitive.head()

,store,mcd_1_mi,subway_1_mi,wendys_1_mi,bk_1_mi,tb_1_mi,mcd_hlf_mi,subway_hlf_mi,wendys_hlf_mi,bk_hlf_mi,tb_hlf_mi
0,5938,0,0,0,0,0,0,0,0,0,0
1,6394,0,0,1,1,0,0,0,1,1,0
2,7724,1,2,0,0,0,1,2,0,0,0
3,6204,1,1,1,1,1,0,1,0,1,0
4,6830,1,2,1,1,1,1,1,1,1,1


In [20]:
competitive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2183 entries, 0 to 2182
Data columns (total 11 columns):
store            2183 non-null int64
mcd_1_mi         2183 non-null int64
subway_1_mi      2183 non-null int64
wendys_1_mi      2183 non-null int64
bk_1_mi          2183 non-null int64
tb_1_mi          2183 non-null int64
mcd_hlf_mi       2183 non-null int64
subway_hlf_mi    2183 non-null int64
wendys_hlf_mi    2183 non-null int64
bk_hlf_mi        2183 non-null int64
tb_hlf_mi        2183 non-null int64
dtypes: int64(11)
memory usage: 187.7 KB


### restaurant

The *restaurant* data set is a basic store profile with status, relative location, open, remodel, and closure dates where applicable.

Items to consider:

- Multiple stores have a null value for *rest_addrstate*. This indicates there is no known state assigned to the store.
- The SQL database from which these data were exported imputed SQL's minimum date value (1753-01-01) for null date values.

The follow measures were taken in the processing of the *restaurant* data:

1. The CSV was read and stored in a DataFrame
2. Column headers were converted to lower case
3. *rest_remodeldt* was converted from an integer to date object
4. Date objects were converted to a uniform MM/DD/YYYY
5. If a minimum date type were presented, it was replaced with *None* type
6. *rest_number* was converted from a float to integer value

In [21]:
restaurant = pd.read_csv('restaurant.csv')

In [22]:
restaurant.columns = map(str.lower, restaurant.columns)

In [23]:
restaurant.head()

,rest_number,rest_status,rest_addrstate,rest_remodeldt,rest_opendate,rest_closedate
0,776.0,Open,IN,17530101,6/14/1976,1753-01-01
1,791.0,Open,NC,20081209,3/15/1977,1753-01-01
2,808.0,Open,CA,20181006,11/29/1975,1753-01-01
3,822.0,Open,NE,20180517,2/16/1976,1753-01-01
4,841.0,Open,FL,19990301,1/14/1978,1753-01-01


In [24]:
restaurant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5355 entries, 0 to 5354
Data columns (total 6 columns):
rest_number       5354 non-null float64
rest_status       5354 non-null object
rest_addrstate    4887 non-null object
rest_remodeldt    5355 non-null int64
rest_opendate     5355 non-null object
rest_closedate    5355 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 251.1+ KB


Date values should be converted to a uniform type.

In [25]:
restaurant['rest_remodeldt'] = pd.to_datetime(restaurant['rest_remodeldt'].astype(str), format='%Y%m%d')

**Note**: The *rest_remodeldt* must be converted to a date object from an integer.

In [26]:
for i in ['rest_remodeldt', 'rest_closedate', 'rest_opendate']:
    restaurant[i] = pd.to_datetime(restaurant[i]).dt.strftime('%m/%d/%Y')

for i in ['rest_remodeldt', 'rest_closedate']:
    restaurant.loc[restaurant[i] == '01/01/1753', i] = None

**Note**: When the CSV files were exported from the SQL database they were stored in, the null values carried the database's native, minimum date value with them (1753-01-01). This portion removes those values.

In [27]:
restaurant['rest_number'] = restaurant['rest_number'].astype('Int64')

**Note**: The *rest_value* number should be converted to an integer for uniformity.

In [28]:
restaurant.head()

,rest_number,rest_status,rest_addrstate,rest_remodeldt,rest_opendate,rest_closedate
0,776,Open,IN,None,06/14/1976,None
1,791,Open,NC,12/09/2008,03/15/1977,None
2,808,Open,CA,10/06/2018,11/29/1975,None
3,822,Open,NE,05/17/2018,02/16/1976,None
4,841,Open,FL,03/01/1999,01/14/1978,None


In [29]:
restaurant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5355 entries, 0 to 5354
Data columns (total 6 columns):
rest_number       5354 non-null Int64
rest_status       5354 non-null object
rest_addrstate    4887 non-null object
rest_remodeldt    1879 non-null object
rest_opendate     5355 non-null object
rest_closedate    1808 non-null object
dtypes: Int64(1), object(5)
memory usage: 256.4+ KB


### store_print

This dataset required no processing aside from reading and storing in a DataFrame.

What the values represent is not entirely clear.

In [30]:
store_print = pd.read_csv('store_print_info.csv')

In [31]:
store_print.head()

,rest_number,drop_date,fiscal_year,fiscal_week_number
0,10,3/24/2015,2015,13
1,1,3/24/2015,2015,13
2,86,3/24/2015,2015,13
3,228,9/29/2015,2015,40
4,85,3/24/2015,2015,13


In [32]:
store_print.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40380 entries, 0 to 40379
Data columns (total 4 columns):
rest_number           40380 non-null int64
drop_date             40380 non-null object
fiscal_year           40380 non-null int64
fiscal_week_number    40380 non-null int64
dtypes: int64(3), object(1)
memory usage: 1.2+ MB


### weather



The values for snow, thunderstorm, rain, and clear_weather are boolean indicating the presence of each. There is no measurement to the degree of that presence.

The first three datasets (weather_2015, weather_2016, and weather_2017) have matching columns and data types. The fourth dataset (weather_2018) has matching columns but mismatched data types for some columns.

The weather_2018 dataset must have matching datatypes before being appended to the other three.

In [33]:
weather = pd.concat(map(pd.read_csv, ['weather_2015.csv', 'weather_2016.csv', 'weather_2017.csv']))
weather.columns = map(str.lower, weather.columns)
for i in ['rest_number', 'rest_dma_code', 'snow', 'thunderstorm', 'rain', 'clear_weather']:
    weather[i] = weather[i].astype('Int64')

wr_2018 = pd.read_csv('weather_2018.csv')
wr_2018.columns = map(str.lower, wr_2018.columns)

for i in ['rest_number', 'rest_dma_code', 'snow', 'thunderstorm', 'rain', 'clear_weather']:
    wr_2018[i] = wr_2018[i].astype('Int64')

In [34]:
weather = wr_2018.append(weather, ignore_index=True, sort=False)
weather = weather.reset_index(drop=True)

In [35]:
weather.head()

,rest_number,rest_dma_code,rest_dma_desc,date_ty,temp_avg,temp_max,temp_min,thunderstorm_bucket,rain_bucket,snow_bucket,snow,thunderstorm,rain,clear_weather
0,1,536,YOUNGSTOWN,1/14/2018,9.872857,24.53,-9.67,NaN,NaN,NaN,0,0,0,1
1,1,536,YOUNGSTOWN,3/26/2018,42.015714,53.33,28.13,NaN,NaN,NaN,0,0,0,1
2,1,536,YOUNGSTOWN,3/7/2018,35.072857,40.73,29.93,NaN,mild,moderate,1,0,1,0
3,1,536,YOUNGSTOWN,5/11/2018,51.787142,58.73,42.53,NaN,mild,NaN,0,0,1,0
4,1,536,YOUNGSTOWN,2/24/2018,42.658571,47.93,37.13,NaN,moderate,NaN,0,0,1,0


In [36]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1678689 entries, 0 to 1678688
Data columns (total 14 columns):
rest_number            1645006 non-null Int64
rest_dma_code          1645006 non-null Int64
rest_dma_desc          1645006 non-null object
date_ty                1645006 non-null object
temp_avg               1645006 non-null float64
temp_max               1645006 non-null float64
temp_min               1645006 non-null float64
thunderstorm_bucket    175039 non-null object
rain_bucket            675064 non-null object
snow_bucket            150291 non-null object
snow                   1645006 non-null Int64
thunderstorm           1645006 non-null Int64
rain                   1645006 non-null Int64
clear_weather          1645006 non-null Int64
dtypes: Int64(6), float64(3), object(5)
memory usage: 188.9+ MB


In [37]:
weather.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
rest_number,1645006.0,4707.706301,2999.923875,1.000000,1211.000000,5934.000000,7317.00,9002.00
rest_dma_code,1645006.0,563.754927,63.872583,501.000000,518.000000,538.000000,588.00,770.00
temp_avg,1645006.0,58.655499,19.419261,-20.341428,44.458571,62.715714,74.93,94.73
temp_max,1645006.0,67.458183,20.364944,-18.670000,53.330000,71.330000,83.93,125.33
temp_min,1645006.0,47.206496,18.732557,-29.470000,33.530000,49.730000,62.33,91.13
snow,1645006.0,0.091362,0.288123,0.000000,0.000000,0.000000,0.00,1.00
thunderstorm,1645006.0,0.106406,0.308357,0.000000,0.000000,0.000000,0.00,1.00
rain,1645006.0,0.419164,0.493422,0.000000,0.000000,0.000000,1.00,1.00
clear_weather,1645006.0,0.653525,0.475847,0.000000,0.000000,1.000000,1.00,1.00


## Conclusion

### Call to Action

## References

## Legends

### Figures

### Tables